# ETAS-based modeling for infectious desease spreading

## Notations:

1. Infection event $i$, denoted as $(t_i,\mathbf{x}_i,N_i)$, corresponds to the $i$th earthquake (including aftershock) in ETAS model, denoted as $(t_i,\mathbf{x}_i,M_i)$, where $\mathbf{x}_i$ and $t_i$ represents the spatio-temporal location of the infection event or earthquake $i$; $N_i$ is the number of infected people during infection event $i$, which is similar to the $i$th earthquake magnitude $M_i$.

2. In ETAS model, the rate of earthquake occurrence at time $t$ depends on the entire history of earthquakes before the current moment $t$, $H_e=\{(t_i,\mathbf{x}_i,M_i)|t_i<t\}$, as follows:

$$\lambda_e(t|H_e)=\mu+\sum_{i:t_i<t}g_e(t-t_i)h_e(M_i),$$

in which

$$ g_e(t) = \frac{1}{(t+c)^p}, $$

$$ h_e(M) = k_e e^{\alpha_e(M-M_0)}, $$

where, $\mu$ is the background occurance rate of earthquakes, $g_e(t)$ is the temporal decay of an earthquake influence, $h_e(M)$ is the spatial influence according to its magnitude; $c$ and $p$ are parameters adjusting $g_e(t)$, $k_e$, $\alpha_e$ and $M_0$ are parameters adjusting $h_e(M)$. $g_e(t)$ and $h_e(M)$ corresponds to the statistial patterns in seismology, known as Omori’s law (occurance rate is inverse proportional to $t$) and Gutenberg-Richter law (occurance rate is proportional to exponential magnitude), respectively.

3. Analogous to ETAS model, we can reasonably assume the occurrence rate of infection events is also determined by the entire infection history before current moment $t$, $H_d=\{(t_i,\mathbf{x}_i,N_i)|t_i<t\}$, as follows:

$$\lambda_d(t|H_d)=\sum_{i:t_i<t}g_d(t-t_i)h_d(N_i,\mathbf{x},t),$$

in which

$$ g_d(t) = \frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{1}{2}(\frac{t-\tau}{\sigma})^2}, $$

$$ h_d(N) = k_d(\frac{N}{N_0})^{\alpha_d}, $$

Notice the differences between $g_d$ and $g_e$, as well as between $h_d$ and $h_e$. $g_d$ is a gaussian distribution instead of inverse proportional along time, since the average infectability of an infected individual varying along time should experience the incubation (zero to low infectability), onset of symptoms (low to high infectability), and recovery (high to low eventually to zero infectability) or quarntine/desease (high to zero infectability by cutoff). $h_d$ is essentially the same format as $h_e$, since $M_i$ as the magnitude represents the earthquake amplitude in log scale, whereas $N_i$ is just the infected number of event $i$ in regular scale.

Due to the resolution limit, we assume all infection events happening within one time interval and one spatial grid are viewed as one infection event, where the time and spatial location are represented by discretized temporal and spatial indices, and the infected number is the total number of infected within this time interval and spatial grid. Hence, we can determine the number of infection events at $t$ as $\lambda_d(t|H_d)$ using the infection history, and this event number equals to the grid number that being infected at $t$.

4. Apart from infection events, the expected infection distribution at $t$ is expressed as

$$ n_I(\mathbf{x},t|H_d)=R_0(t)(1-I(\mathbf{x},t))\sum_{i:t_i<t}N_ig_d(t-t_i)f(\mathbf{x},\mathbf{x}_i,t), $$

in which

$$f(\mathbf{x},\mathbf{x}_i,t)=w_G(t)G_n(|\mathbf{x}-\mathbf{x}_i|,\sigma_d(t))+(1-w_G(t))P_n(\mathbf{x},\mathbf{x}_i).$$

where, $R_0(t)$ is the basic reproduction number, $I(\mathbf{x},t)$ is the immunity ratio, $g_n(t)$ and $f(\mathbf{x},\mathbf{x}_i,t)$ are the temporal and spatial PDFs of infection, respectively. $g_n(t)$ is simply the normalized $g_e(t)$, $f(\mathbf{x},\mathbf{x}_i,t)$ is composed by two parts: the 2-D Gaussian function $G_n(|\mathbf{x}-\mathbf{x}_i|,\sigma_d,t)$ representing the short-distance wandering, while $P_n(\mathbf{x},\mathbf{x}_i)$ representing the long-distance traveling, $w_G(t)$ is the weight for the short-distance wandering along time.

Thus, the total expected infection number at $t$ is

$$ N_I(t|H_d) = \int_{\mathbf{x}}n_I(\mathbf{x},t|H_t)d\mathbf{x}$$

5. To simulate the infection events at $t$, we first sample $n_e = \mathrm{round}(\lambda_d(t))$ spatial positions according to the spatial PDF as normalized $n_I(\mathbf{x},t)$. Then we assign $N_i$ to the sampled $n_e$ positions, as $n_e$ infection events:

$$ N_i=\mathop{\min}[N_I(t)\frac{n_I(\mathbf{x}_i,t)}{\sum_i^{n_e}n_I(\mathbf{x}_i,t)},N_s(\mathbf{x}_i,t)]$$

where, $N_s(\mathbf{x}_i,t)=N(\mathbf{x}_i)(1-I(\mathbf{x}_i,t))$ is the susceptible population at location $\mathbf{x}_i$ and time $t$; $N(\mathbf{x}_i)$ is the static population at position $\mathbf{x}_i$.

## Required data and prior information

1. The open-source data in singapore include the imported case number at each day, confirmed case number at each day $N_I(t)$ (in community, which is the target we are trying to simulate). However, there is no specific locations for each confirmed case.

In [ ]:
%load_ext autoreload
%autoreload 2
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as pcl
import matplotlib.cm as cm
from ETAS_fun import pdegconv, gd, fxxi, infevnt, ifds_ETAS, dispift
from Hd_geninv import rdHdgen, invka
from R0_inv import R0inv

In [ ]:
# readin all available data (from 1, Jan to 12, Nov 2020)
path = './resources'
outpath = './outputs'
sgdata = 'Singapore_case_No.xlsx'
NI = pd.read_excel(f'{path}/{sgdata}') # all data (https://www.moh.gov.sg/covid-19/situation-report)

In [ ]:
NI

In [ ]:
# select required data
NI_c = np.array(NI['Community']) # total community cases
NI_ids = np.array(NI['detected through surveillance (imported)']) # total imported cases (not being isolated until 5 days after onset of their symptoms)
nt = len(NI_c) # total days

In [ ]:
# smooth the data
ns = 7
s = np.ones((ns))/ns
NIs_ids = pdegconv(NI_ids,s) # smoothed community infected number
NIs_c = pdegconv(NI_c,s) # smoothed imported infected number

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(NI_c,'b:',label='community')
ax.plot(NIs_c,'b',label='smoothed community')
ax.plot(NI_ids,'r:',label='imported')
ax.plot(NIs_ids,'r',label='smoothed imported')
ax.set_xlabel('Time (day)')
ax.set_ylabel('Infected number ($N_I$)')
_ = ax.legend()

In [ ]:
fig.savefig(f'{outpath}/NI.png',dpi=300)

2. Prior knowledge for $g_d(t)$ and $f(\mathbf{x},\mathbf{x}_i,t)$

(1) According to current COVID-19 study, we can have an average $g_d(t)$: $\tau$ and $\sigma$ are determined according to the average course of COVID-19. These parameters are assumed constant throughout the modeling peroid.

In [ ]:
# compose gd as gt here
t_onset = 4 # the onset of symptoms
t_peak = 7.5 # the peak symptom
t_cut = t_onset+5 # average cutoff day is 5 days after the onset of symptoms
t = np.arange(t_peak*2)
gt = gd(t,t_peak,t_peak/3,cf=t_cut) # sigma is approximated t_peak/3 days

In [ ]:
# plot gt
T = np.arange(t_peak*2)
gtd = gd(t,t_peak,t_peak/3)
gtd = gtd/np.amax(gtd)*np.amax(gt)
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(gtd[:t_cut+1],'b')
ax.plot(T[t_cut:],gtd[t_cut:],'b--')
ax.plot([t_onset,t_onset],[-0.01,0.2],'r')
ax.plot([t_cut,t_cut],[-0.01,0.2],'r')
ax.set_ylim(0,0.2)
ax.set_xticks(np.linspace(0,t_peak*2,6))
ax.set_yticks(np.linspace(0,0.2,5))
ax.set_xlabel('Time (day)')
ax.text(t_onset*0.6,0.15,'''Onset of symptoms''',fontsize=12)
ax.text(t_cut*0.9,0.17,'''Quarantined''',fontsize=12)
_ = ax.set_ylabel('Infectability ($g_d$)')

In [ ]:
fig.savefig(f'{outpath}/gd.png',dpi=300)

(2) $G_n(|\mathbf{x}-\mathbf{x}_i|,\sigma_d(t))$ represents the short-distance wandering in the neighborhood, whereas $P_n(\mathbf{x},\mathbf{x_i})$ represents the long-distance travelling to anywhere in the modeling area. $w_G(t)$ controls the ratio between these two kinds of human movement.

(3) Here, we need two types of information for creating $f(\mathbf{x},\mathbf{x}_i,t)$: general demography and quantified containment measures.

In [ ]:
dmgfn = 'dmg.dat'
efffn = 'effarea.dat'
stgnb = 'sg_sidx.xlsx' #https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker

In [ ]:
# readin demographic data and containment stringency index
dmg = np.fromfile(f'{path}/{dmgfn}',dtype=np.float64)
effarea = np.fromfile(f'{path}/{efffn}',dtype=bool)
stgnb = pd.read_excel(f'{path}/{stgnb}') # stringency index

In [ ]:
# reshape the data
shape = (235,400)
dmg = np.reshape(dmg,shape)
effarea = np.reshape(effarea,shape)

In [ ]:
# smooth the stringency index
sidx = np.array(stgnb['Singapore'])
sidx[0] = sidx[1] # eliminate the first 0 index
ss = pdegconv(sidx,s)

In [ ]:
# plot the stringency index
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(sidx/100,label='original')
ax.plot(ss/100,label='smoothed')
ax.set_xlabel('Time (day)')
ax.legend()
_ = ax.set_ylabel('s')

In [ ]:
fig.savefig(f'{outpath}/sindx.png',dpi=300)

(4) $f(\mathbf{x},\mathbf{x_i},t)$: $w_G(t)$, $\sigma_d(t)$ and $P_n(\mathbf{x},\mathbf{x}_i)$ are predetermined according to human transportation. We assume $P_n(\mathbf{x},\mathbf{x}_i)$ does not change along time, whereas $w_G(t)$ and $\sigma_d(t)$ do vary with time, which can reflect the containment measures, e.g., during "circuit breaker" in Singapore, $w_G(t)$ increases while $\sigma_d(t)$ decreases, since huamn movement is restricted into smaller area.

i. the highest stringency parts (>0.7) corresponds to "circuit breaker" and its reopening phase 1 and 2;

ii. followed by phase 2, the stringency index keeps around 0.5 for the phase 3.

In [ ]:
ssN = ss/100 # normalize ss into [0,1] range
# wG: linear mapping from [0.2,0.9] to [0,0.9^2] in sidx^2
wG = (ssN/0.9)**2*(0.9-0.2)+0.2
# (sigma_d)sd: linear mapping from [10,1] to [0,0.9^4] in sidx^4
sd = (ssN/0.9)**4*(1-10)+10

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(12,10))
ax[0].plot(wG)
ax[1].plot(sd)
for i in range(2):
    ax[i].set_xlabel('Time (day)')
ax[0].set_ylabel('$w_G$')
_ = ax[1].set_ylabel('$\sigma_d$')

(5) $P_n$ is the background PDF for human transportation. Here we simply use the logscaled demography. 
This indicates that people tends to travel to densely populated area since those regions are where most 
working spaces, schools, shopping malls and food courts are.

In [ ]:
# log dmg to obtain background pdf
dmg1 = dmg
dmg1[dmg1<1] = 1 # avoid log(0)
bkd = np.log(dmg1)
bkd = bkd/np.sum(bkd) # logscaled dmg as background pdf

In [ ]:
# plot Pn
Vmin = 0
Vmax = 0.000035
cN = pcl.Normalize(vmin=Vmin, vmax=Vmax)
fig,ax = plt.subplots(1,1,figsize=(9.5,4))
ax.imshow(bkd,cmap='viridis')
x1 = 300
x2 = 380
x3 = 315
y1 = 220
y2 = 215
y3 = 210
ax.set_xticks([])
ax.set_yticks([])
ax.plot([x1,x2],[y1,y1],'r',linewidth=3)
ax.plot([x1,x1],[y1,y2],'r',linewidth=3)
ax.plot([x2,x2],[y1,y2],'r',linewidth=3)
cb = fig.colorbar(cm.ScalarMappable(norm=cN, cmap='viridis'))
ctick = np.linspace(Vmin,Vmax,6)
cb.set_ticks(ctick)
cb.set_ticklabels([f'{i*1e5:.1f}' for i in ctick])
ax.text(420,-10,'x1e-5')
_ = ax.text(x3,y3,'10 km',color='r',fontsize=15)

In [ ]:
BB = ax.get_position()
BB.x0 = 1.8
BB.y0 = 0.35
BB.x1 = 8
BB.y1 = 3.8

In [ ]:
fig.savefig(f'{outpath}/Px.png',dpi=300,bbox_inches=BB)

## Parameter fitting using synthetic infection event history

If we could get $H_d$, we are able to fit a $h_d(N)$. Then $R_0(t)$ is fitted according to $N_I(t)$.

i. $h_d(N)$: $N_0$ is the infection number threshold that being considered as an infection event. In singapore, since the total case is small, we set $N_0=1$ throughout the modeling. $k_d$ and $\alpha_d$ are fitted according to infection event history $H_d$. When modeling in a relatively homogeneous area, e.g., a relatively small country like singapore, we assume $k_d$ and $\alpha_d$ are constant in space. As for temporal variation, we assume they are also constant.

ii. $R_0$: After above parameters are determined, we can fit for $R_0$ based on the confirmed case No. $N_I(t)$. 

### Create $H_d$ randomly according to real $N_I(t)$

1. In view of Singapore's situation, we assume each infection event involve $n_e$ people, where $n_e\in U(1,10)$.

2. Based on the real $N_I(t)$, we can randomly generate $n_e$ (and sample corresponding positions) until they sum to $N_I(t)$. Then, we can obtain $\lambda_d(t)$.

In [ ]:
Hd_c = rdHdgen(NI_c,dmg) # community infection events
Hd_i = rdHdgen(NI_ids,dmg,Hd_c) # imported infection events
Hd = copy.deepcopy(Hd_c)
Hd.evnt_add(Hd_i.indx,Hd_i.indt,Hd_i.Ni) # total infection events

In [ ]:
# calculate infection event number varying with time and infected number for each event
Ldc,HdNic = Hd_c.LdHdNi_cal(nt)
Ldi,HdNii = Hd_i.LdHdNi_cal(nt)
Ld,HdNi = Hd.LdHdNi_cal(nt)

In [ ]:
# plot statistical information of Hd
fig,ax = plt.subplots(1,2,figsize=(17,5))
ax[0].plot(Ldc,label='community')
ax[0].plot(Ldi,label='imported')
#ax[0].plot(Ld,label='total')
ax[0].set_xlabel('Time (day)')
ax[0].set_ylabel('$\lambda_d$')
ax[1].hist([HdNic,HdNii],bins=np.arange(11)+1,rwidth=0.5,label=['community','imported'])
ax[1].set_xlabel('Infected number ($N_i$)')
ax[1].set_ylabel('Infection event number')
xtick = np.arange(10)+1.5
ax[1].set_xticks(xtick)
ax[1].set_xticklabels([f'{int(i-0.5)}' for i in xtick])
for i in range(2):
    ax[i].legend()

In [ ]:
fig.savefig(f'{outpath}/Hdstat.png',dpi=300)

In [ ]:
# calculate the infected number NI(t) varying with time
tr = [i for i in range(nt)]
NIc = Hd_c.NIcal(tr) # NIc should equal to NI_c
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(NIc,label='Generated')
ax.plot(NI_c,label='Recorded')
ax.set_xlabel('Time (day)')
ax.set_ylabel('Infected number')
ax.legend()
# calculate the final immunity
I = Hd.Ical(dmg,nt)
print(f'maximum immunity ratio must smaller than 1: {np.amax(I)}')

In [ ]:
# display all synthectic infection events
fx = fxxi(shape, effarea, bkd)
disp = dispift(fx,outpath=outpath)

In [ ]:
disp.dispevnt_history(nt,Hd_c,Hd_i)

### Fit for $k_d$ and $\alpha_d$: $\mathop{\min}_{k_d,\alpha_d} \lVert \mathbf{GH}(k_d,\alpha_d)-\mathbf{\lambda} \rVert_2^2$

In [ ]:
ive = invka(gt,Ld,HdNi,Ldc,N0=1)
r'''Notice the input infection event history is the total infection events, including community and imported, 
however, the fitted infection event number only contain the community cases, since the imported cases are not 
influenced by the community cases.'''

In [ ]:
k0 = 1
a0 = 1
Ld0 = ive.testka(k0,a0)

In [ ]:
k,a = ive.iterupdate(k0,a0,Nk=0.01,Na=0.01,tol=1e-5)

In [ ]:
print(f'k={k}')
print(f'a={a}')

In [ ]:
Lde = ive.testka(k,a)

In [ ]:
label1 = 'predicted infection event number (initial)'
label2 = 'predicted infection event number (inverted)'

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(ive.Ldy,label='infection event number')
ax.plot(Ld0,'r--',label=label1)
ax.plot(Lde,'r',label=label2)
ax.set_xlabel('Time (day)')
ax.set_ylabel('$\lambda_d$')
ax.text(200,20,f'initial: $(k_d={k0},\\alpha_d={a0})$')
ax.text(200,15,f'inverted: $(k_d={k:.2f},\\alpha_d={a:.2f})$')
ax.legend()

In [ ]:
fig.savefig(f'{outpath}/kafit.png',dpi=300)

### Fit for $R_0(t)$:

1. We know that $R_0(t)$ should be affected by the containment measures. Higher the stringency index is, lower the $R_0(t)$ is. Similar to $\lambda_d(t)$, we assume the relationship between $R_0(t)$ and stringency index $s(t)$ is

$$R_0(t) = ls(t)^{\beta},$$

where, $l>0$ and $\beta<0$.

2. Now we can use the infection event history to invert for $l$ and $\beta$ so that a relationship between $R_0(t)$ and $s(t)$ can be estabilished.

$$\mathop{\min}_{(l,\beta)}\lVert\mathbf{FR}_0(k,\beta)-\mathbf{N}_I\rVert_2^2,$$

in which

$$\mathbf{FR}_0(k,\beta)=R_0(t|k,\beta)\int(1-I(\mathbf{x},t))\sum_{i:t_i<t}N_ig_n(t-t_i)f(\mathbf{x},\mathbf{x}_i,t)d\mathbf{x}$$

In [ ]:
from R0lb_inv import lbinv
NI_y = Hd_c.NIcal(tr) # the NI being fitted is the community case No.
lbINV = lbinv(dmg,Hd,ssN,gt,fx,sd,wG,NI_y)
l0 = 1
b0 = -1 # initial solutions
NI0 = lbINV.testlb(l0,b0) # initial modeling result

In [ ]:
l,b = lbINV.iterupdate(l0,b0)
print(f'l={l}')
print(f'b={b}')
NIe = lbINV.testlb(l,b) # inverted modeling result

In [ ]:
label1 = 'predicted infected number (initial)'
label2 = 'predicted infected number (inverted)'

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(lbINV.NI,label='infected number')
ax.plot(NI0,'r--',label=label1)
ax.plot(NIe,'r',label=label2)
ax.set_xlabel('Time (day)')
ax.set_ylabel('$N_I$')
ax.text(200,40,f'initial: $(l={l0},\\beta={b0})$')
ax.text(200,33,f'inverted: $(l={l:.2f},\\beta={b:.2f})$')
_ = ax.legend()

In [ ]:
fig.savefig(f'{outpath}/lbfit.png',dpi=300)

In [ ]:
R0 = lbINV.R0cal(l,b)
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(R0)
ax.set_xlabel('Time (day)')
ax.set_ylabel('$R_0$')
_ = ax.text(150,1.6,f'$R_0={l}\\times s(t)^{b}$',fontsize=15)

In [ ]:
fig.savefig(f'{outpath}/R0.png',dpi=300)

## Modeling from 1 Jan, 2020 to 12 Nov, 2020 using fitted parameters and the imported case number

In [ ]:
mod = ifds_ETAS(dmg,gt,1,k,a,fx)

In [ ]:
nIt = mod.formod(R0,sd,wG,Hdi=Hd_i)

In [ ]:
NIm = mod.Hd.NIcal(tr)
NIi = Hd_i.NIcal(tr)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(NI_c,'r',label='real community cases')
ax.plot(NIm-NIi,'b',label='modeled community cases')
ax.set_xlabel('Time (day)')
ax.set_ylabel('$N_I$')
ax.legend()

In [ ]:
fig.savefig(f'{outpath}/NImod.png',dpi=300)

In [ ]:
# delete imported cases from mod.Hd
Hdm = copy.deepcopy(mod.Hd)
Hdm.evnt_del(Hd_i)
NIm2 = Hdm.NIcal(tr)
print(f'Difference between NIm and NIm2: {np.sum(NIm2-(NIm-NIi))}')

In [ ]:
disp = dispift(fx,dpi=40,outpath=outpath)
# plot nI and Hd evolving with time
disp.dispevnt_history(nt,Hdm,Hd_i)

In [ ]:
for i in range(nt):
    nI = nIt[i]
    indx,Ni = mod.Hd.evnt_t(i)
    disp.dispnI(i,nI,indx,Ni)

## What if there is no "circuit breaker"

In [ ]:
sdn = np.array(sd)
sdn[75:] = sd[75]
wGn = np.array(wG)
wGn[75:] = wG[75]
R0n = np.array(R0)
R0n[75:] = R0[75]

In [ ]:
mod = ifds_ETAS(dmg,gt,1,k,a,fx)

In [ ]:
nIt = mod.formod(R0n,sdn,wGn,Hdi=Hd_i)

In [ ]:
NIm = mod.Hd.NIcal(tr)
NIi = Hd_i.NIcal(tr)

In [ ]:
# delete imported cases from mod.Hd
Hdm = copy.deepcopy(mod.Hd)
Hdm.evnt_del(Hd_i)
NIm2 = Hdm.NIcal(tr)
print(f'Difference between NIm and NIm2: {np.sum(NIm2-(NIm-NIi))}')

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,4))
ax.plot(NI_c,'r',label='real community cases')
ax.plot(NIm-NIi,'b',label='modeled community cases')
ax.set_xlabel('Time (day)')
ax.set_ylabel('$N_I$')
ax.legend()

In [ ]:
fig.savefig(f'{outpath}/NImod_new.png',dpi=300)

In [ ]:
disp = dispift(fx,dpi=40,outpath=outpath)
# plot nI and Hd evolving with time
disp.dispevnt_history(nt,Hdm,Hd_i)

In [ ]:
disp = dispift(fx,dpi=38,outpath=outpath)
for i in range(nt):
    nI = nIt[i]
    indx,Ni = mod.Hd.evnt_t(i)
    disp.dispnI(i,nI,indx,Ni)